In [1]:
# ARTIST INFO CODIGO LIMPIO 25 FEB 2025

In [2]:
import pandas as pd
import requests
import json
import re  # Importar el módulo para limpiar HTML

/Users/isabelgarcia/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
api_key = "XXXXXXXXXXXXXXXXXXXXXX"
share_secret = "XXXXXXXXXXXXXXXXXX"


- Sacamos informacion del artista desde last.fm y nos lo muestra en un DataFrame.
- Información extraida: nombre, url, reproducciones, artistas similares, generos y resumen de la biografia.

In [5]:
#  Lista de los 5 artistas más populares de la playlist de Eurovisión 2022
lista_artistas_eurovision = ["Mahmood", "LUM!X", "S10", "MARO", "Subwoolfer"]

In [6]:
# Primero hago la funcion para un unico artista: 

def obtener_info_artista(artist_name, api_key):  #esta funcion para sacar info del artista con ARTIST_NAME y API_KEY: devuelve un diccionario con sus datos.


    # Endpoint de la API de Last.fm
    url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artist_name}&api_key={api_key}&format=json"
    
    # Hago la llamada a la API
    response = requests.get(url)  # hago la llamada a la api.
    info_artist = response.json()  # y la respuesta de la llamada la convierto en json.

    # compruebo la respuesta de la API
    if "error" in info_artist:
        print(f"Error en la API: {info_artist['message']}")
        return pd.DataFrame()   # si hay error dime el error y dame un DATAFRAME vacio.

    # información del artista
    nombre = info_artist["artist"].get("name")
    url = info_artist["artist"].get("url") 
    oyentes = info_artist["artist"]["stats"].get("listeners")
    reproducciones = info_artist["artist"]["stats"].get("playcount")

    #resumen de la biografía y limpiar etiquetas HTML
    bio_summary = info_artist["artist"]["bio"].get("summary")
    bio_summary_clean = re.sub(r"<.*?>", "", bio_summary)  # Eliminar etiquetas HTML

    # géneros musicales
    generos = [tag["name"] for tag in info_artist["artist"]["tags"]["tag"][:3]] # limito a tres generos del artista
    generos_texto = ", ".join(generos)  # utilizo este codigo para que aparezca como cadena de texto y no como lista con corchetes y comillas.

    #artistas similares
    artistas_similares = [artista["name"] for artista in info_artist["artist"]["similar"]["artist"][:3]]  # limito a tres artistas similares
    artistas_similares_texto = ", ".join(artistas_similares)   # igual que en genero lo convierto en cadena de texto.

    return {                    # me devuelve un diccionario con todos los datos que le he pedido del artista.
        "artista": nombre,
        "url":url,
        "oyentes": oyentes,
        "reproducciones": reproducciones,
        "generos": generos_texto,
        "artistas_similares": artistas_similares_texto,
        "biografia_resumen": bio_summary_clean
    }


#  la información de VARIOS artistas y la almacena en un DataFrame.

def obtener_info_varios_artistas(lista_artistas, api_key):
   
    datos_artistas = []

    for artista in lista_artistas:
        info = obtener_info_artista(artista, api_key)  # llamo a la funcion para obtener los datos para cada artista, que itere
        if info:  # Solo agregar si la consulta fue ok
            datos_artistas.append(info)

    
    df_artistas = pd.DataFrame(datos_artistas)   # Creo DataFrame con todos los artistas
    return df_artistas   # return el DataFrame de todos los artistas.
    
"""df_artistas_eurovision = obtener_info_varios_artistas(lista_artistas_eurovision, api_key)

print(df_artistas_eurovision.head())"""  #lo inhabilito porque es otra forma de sacar el dataframe pero se ve peor.

# DataFrame con la información de todos los artistas truco de CHATGPT
from IPython.display import display  # Importar display para Jupyter

df_artistas_eurovision = obtener_info_varios_artistas(lista_artistas_eurovision, api_key)
display(df_artistas_eurovision)  # ✅ Mejor visualización en Jupyter



,artista,url,oyentes,reproducciones,generos,artistas_similares,biografia_resumen
0,Mahmood,https://www.last.fm/music/Mahmood,213811,8652431,"pop, italian, rnb","Elodie, Angelina Mango, Rose Villain",Alessandro Mahmoud (born 12 September 1992 in ...
1,LUM!X,https://www.last.fm/music/LUM%21X,202859,2598088,"dance, austrian, Eurovision","R3hab, Tungevaag, B3nte",LUM!X is an Austrian DJ and producer. Luca Mic...
2,S10,https://www.last.fm/music/S10,85114,3568678,"pop, dutch, seen live","Froukje, Merol, Cornelia Jakobs","Stien den Hollander, better known as S10, is a..."
3,Maro,https://www.last.fm/music/Maro,183131,2769510,"pop, portuguese, Eurovision","Iolanda, Salvador Sobral, Cláudia Pascoal",There are two artists that go by the name Maro...
4,Subwoolfer,https://www.last.fm/music/Subwoolfer,66905,900344,"pop, Eurovision Song Contest, norwegian","KEiiNO, THE ROOP, Efendi",Subwoolfer will be competing in the Melodi Gra...


In [8]:
# Para convertirlo a CSV.

df_artistas_eurovision.to_csv("artistas_eurovision_2022.csv", index=False)
print("✅ Archivo guardado: artistas_eurovision_2022.csv")

✅ Archivo guardado: artistas_eurovision_2022.csv
